<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import pandas as pd
from janitor import clean_names

In [135]:
from os import name
import requests
from bs4 import BeautifulSoup
import re
import json

In [136]:
def scrape_url(url):

    with open('data/user_agent.json') as json_file:
        headers = json.load(json_file)['headers']

    response = requests.get(url, headers=headers)

    soup = BeautifulSoup(response.text, features='html.parser')
    return soup


def scrape_article_text():

    url = 'https://gdpr-info.eu/'

    soup = scrape_url(url)

    links = soup.find_all(name='table')

    anchors = [table.find_all('a') for table in soup.find_all('table')][0]


    all_articles = {}
    # loop through anchors
    
    for anchor in anchors:
        if len(anchor.get_text()) > 8:
            current_chap = anchor.get_text()
            all_articles[current_chap] = {}
            all_articles[current_chap]['title'] = anchor.get('data-title')
            all_articles[current_chap]['link'] = anchor.get('href')
            all_articles[current_chap]['articles'] = {}
        elif len(anchor.get_text()) < 4:
            current_art = anchor.get_text()
            all_articles[current_chap]['articles'][current_art] = {}
            all_articles[current_chap]['articles'][current_art]['title'] = anchor.get('data-title')
            link = anchor.get('href')
            art_soup = scrape_url(link)
            try:
                art_text = art_soup.find('div', {'class': 'entry-content'}).find('ol').get_text()
            except:
                art_text = art_soup.find('div', {'class': 'entry-content'}).find('p').get_text()
            all_articles[current_chap]['articles'][current_art]['link'] = link
            all_articles[current_chap]['articles'][current_art]['text'] = art_text

    all_articles

    with open('data/scraped_art_text.json', 'w', encoding='utf-8') as f:
        json.dump(all_articles, f, ensure_ascii=False, indent=4)

    pass

In [138]:
link = 'https://gdpr-info.eu/art-4-gdpr/'
art_soup = scrape_url(link)

In [139]:
art_soup

<!DOCTYPE html>

<html class="no-js" lang="en-US">
<head>
<meta charset="utf-8"/>
<meta content="width=device-width" name="viewport"/>
<link href="http://gmpg.org/xfn/11" rel="profile"/>
<link href="https://gdpr-info.eu/xmlrpc.php" rel="pingback"/>
<!--[if lt IE 9]>
<script src="https://gdpr-info.eu/wp-content/themes/twentyfifteen/js/html5.js"></script>
<![endif]-->
<script>(function(html){html.className = html.className.replace(/\bno-js\b/,'js')})(document.documentElement);</script>
<meta content="index, follow, max-image-preview:large, max-snippet:-1, max-video-preview:-1" name="robots">
<!-- This site is optimized with the Yoast SEO plugin v17.2.1 - https://yoast.com/wordpress/plugins/seo/ -->
<title>Art. 4 GDPR – Definitions - General Data Protection Regulation (GDPR)</title>
<link href="https://gdpr-info.eu/art-4-gdpr/" rel="canonical">
<meta content="en_US" property="og:locale">
<meta content="article" property="og:type"/>
<meta content="Art. 4 GDPR – Definitions - General Data P

In [140]:
link = 'https://gdpr-info.eu/art-2-gdpr/'
art_soup_2 = scrape_url(link)

art_soup_2

<!DOCTYPE html>

<html class="no-js" lang="en-US">
<head>
<meta charset="utf-8"/>
<meta content="width=device-width" name="viewport"/>
<link href="http://gmpg.org/xfn/11" rel="profile"/>
<link href="https://gdpr-info.eu/xmlrpc.php" rel="pingback"/>
<!--[if lt IE 9]>
<script src="https://gdpr-info.eu/wp-content/themes/twentyfifteen/js/html5.js"></script>
<![endif]-->
<script>(function(html){html.className = html.className.replace(/\bno-js\b/,'js')})(document.documentElement);</script>
<meta content="index, follow, max-image-preview:large, max-snippet:-1, max-video-preview:-1" name="robots">
<!-- This site is optimized with the Yoast SEO plugin v17.2.1 - https://yoast.com/wordpress/plugins/seo/ -->
<title>Art. 2 GDPR – Material scope - General Data Protection Regulation (GDPR)</title>
<link href="https://gdpr-info.eu/art-2-gdpr/" rel="canonical">
<meta content="en_US" property="og:locale">
<meta content="article" property="og:type"/>
<meta content="Art. 2 GDPR – Material scope - General 

okay, so sublists have an ol inside an ol

if there's an `ausnahme` it's like definitions.



In [142]:
link = 'https://gdpr-info.eu/art-1-gdpr/'
art_soup_1 = scrape_url(link)

art_soup_1

<!DOCTYPE html>

<html class="no-js" lang="en-US">
<head>
<meta charset="utf-8"/>
<meta content="width=device-width" name="viewport"/>
<link href="http://gmpg.org/xfn/11" rel="profile"/>
<link href="https://gdpr-info.eu/xmlrpc.php" rel="pingback"/>
<!--[if lt IE 9]>
<script src="https://gdpr-info.eu/wp-content/themes/twentyfifteen/js/html5.js"></script>
<![endif]-->
<script>(function(html){html.className = html.className.replace(/\bno-js\b/,'js')})(document.documentElement);</script>
<meta content="index, follow, max-image-preview:large, max-snippet:-1, max-video-preview:-1" name="robots">
<!-- This site is optimized with the Yoast SEO plugin v17.2.1 - https://yoast.com/wordpress/plugins/seo/ -->
<title>Art. 1 GDPR – Subject-matter and objectives - General Data Protection Regulation (GDPR)</title>
<link href="https://gdpr-info.eu/art-1-gdpr/" rel="canonical">
<meta content="en_US" property="og:locale">
<meta content="article" property="og:type"/>
<meta content="Art. 1 GDPR – Subject-ma

usually just entry content

In [144]:
# type 1 - expected. No nested lists
art_soup_1.find('div', {'class':'entry-content'}).find('ol')

<ol>
<li>This Regulation lays down rules relating to the protection of natural persons with regard to the processing of personal data and rules relating to the free movement of personal data.</li>
<li>This Regulation protects fundamental rights and freedoms of natural persons and in particular their right to the protection of personal data.</li>
<li>The free movement of personal data within the Union shall be neither restricted nor prohibited for reasons connected with the protection of natural persons with regard to the processing of personal data.</li>
</ol>

In [172]:
# type 2 - nested. give each nest its own counter
a = (
    art_soup_2
    .find('div', {'class':'entry-content'})
    .find('ol')

)

for child in a.descendants:
    print(child)



<li>This Regulation applies to the processing of personal data wholly or partly by automated means and to the processing other than by automated means of personal data which form part of a filing system or are intended to form part of a filing system.</li>
This Regulation applies to the processing of personal data wholly or partly by automated means and to the processing other than by automated means of personal data which form part of a filing system or are intended to form part of a filing system.


<li>This Regulation does not apply to the processing of personal data:
<ol>
<li>in the course of an activity which falls outside the scope of Union law;</li>
<li>by the Member States when carrying out activities which fall within the scope of <a href="http://eur-lex.europa.eu/legal-content/EN/TXT/HTML/?uri=CELEX:12012M/TXT" rel="noopener" target="_blank">Chapter 2 of Title V of the TEU</a>;</li>
<li>by a natural person in the course of a purely personal or household activity;</li>
<li>b

In [154]:
# type 3 (definitions) - each gets its own article number
(
    art_soup
    .find('div', {'class':'entry-content'})
    .find('ol', {'class':'liste-ausnahme'})
    .get_text()
)

'\n‘personal data’ means any information relating to an identified or identifiable natural person (‘data subject’); an identifiable natural person is one who can be identified, directly or indirectly, in particular by reference to an identifier such as a name, an identification number, location data, an online identifier or to one or more factors specific to the physical, physiological, genetic, mental, economic, cultural or social identity of that natural person;\n‘processing’ means any operation or set of operations which is performed on personal data or on sets of personal data, whether or not by automated means, such as collection, recording, organisation, structuring, storage, adaptation or alteration, retrieval, consultation, use, disclosure by transmission, dissemination or otherwise making available, alignment or combination, restriction, erasure or destruction;\n‘restriction of processing’ means the marking of stored personal data with the aim of limiting their processing in t

In [ ]:
try:
    art_text = art_soup.find('div', {'class': 'entry-content'}).find('ol').get_text()
except:
    art_text = art_soup.find('div', {'class': 'entry-content'}).find('p').get_text()
all_articles[current_chap]['articles'][current_art]['link'] = link
all_articles[current_chap]['articles'][current_art]['text'] = art_text

In [2]:
# Reading the json as a dict
with open('data/scraped_fines.json') as json_data:
    fines = json.load(json_data)

In [3]:
fines_df = (
    pd
    .json_normalize(fines['allItems'])
    .rename(columns ={'articleViolated':'article_violated'})
)

In [4]:
fines_df

,id,picture,name,price,authority,date,controller,article_violated,type,source,summary
0,1,https://www.privacyaffairs.com/wp-content/uplo...,Poland,9380,Polish National Personal Data Protection Offic...,10/18/2019,Polish Mayor,Art. 28 GDPR,Non-compliance with lawful basis for data proc...,https://uodo.gov.pl/decyzje/ZSPU.421.3.2019,No data processing agreement has been conclude...
1,2,https://www.privacyaffairs.com/wp-content/uplo...,Romania,2500,Romanian National Supervisory Authority for Pe...,10/17/2019,UTTIS INDUSTRIES,"Art. 12 GDPR, Art. 13 GDPR, Art. 5 (1) c) GDPR...",Information obligation non-compliance,https://www.dataprotection.ro/?page=A_patra_am...,A controller was sanctioned because he had unl...
2,3,https://www.privacyaffairs.com/wp-content/uplo...,Spain,60000,Spanish Data Protection Authority (AEPD),10/16/2019,Xfera Moviles S.A.,"Art. 5 GDPR, Art. 6 GDPR",Non-compliance with lawful basis for data proc...,https://www.aepd.es/resoluciones/PS-00262-2019...,The company had unlawfully processed the perso...
3,4,https://www.privacyaffairs.com/wp-content/uplo...,Spain,8000,Spanish Data Protection Authority (AEPD),10/16/2019,Iberdrola Clientes,Art. 31 GDPR,Failure to cooperate with supervisory authority,https://www.aepd.es/resoluciones/PS-00304-2019...,Iberdrola Clientes violated Article 13 of the ...
4,5,https://www.privacyaffairs.com/wp-content/uplo...,Romania,150000,Romanian National Supervisory Authority for Pe...,10/09/2019,Raiffeisen Bank SA,Art. 32 GDPR,Failure to implement sufficient measures to en...,https://www.dataprotection.ro/?page=Comunicat_...,Raiffeisen Bank Romania did not observe the ne...
...,...,...,...,...,...,...,...,...,...,...,...
875,876,https://www.privacyaffairs.com/wp-content/uplo...,Spain,56000,Spanish Data Protection Authority (AEPD),09/14/2021,"Vodafone Espana, S.A.U.",Art. 6 (1) GDPR,Non-compliance with lawful basis for data proc...,,The Spanish DPNA (AEPD) has fined Vodafone Esp...
876,877,https://www.privacyaffairs.com/wp-content/uplo...,Spain,56000,Spanish Data Protection Authority (AEPD),09/14/2021,"Vodafone Espana, S.A.U.",Art. 6 (1) GDPR,Non-compliance with lawful basis for data proc...,,The Spanish DPA (also known as AEPD) fined Vo...
877,878,https://www.privacyaffairs.com/wp-content/uplo...,Spain,4000,Spanish Data Protection Authority (AEPD),09/16/2021,Frigorifica Botana S.L.,Art. 5 (1) c) GDPR,Failure to comply with data processing principles,,The Spanish DPA (also known as AEPD) has fined...
878,879,https://www.privacyaffairs.com/wp-content/uplo...,Spain,18000,Spanish Data Protection Authority (AEPD),09/20/2021,CEDICO,Art. 5 (1) f) GDPR,Failure to comply with data processing principles,,"The Spanish DPA (AEPD) has fined CEDICO, CENTR..."


In [7]:
def squeeze(x):
    original_columns = x.index.tolist()

    pre = x.replace({'':np.nan})
    squeezed = pre.dropna()
    squeezed.index = [original_columns[n] for n in range(squeezed.count())]

    return squeezed.reindex(original_columns, fill_value=np.nan).dropna()

In [8]:
art_text = (
    pd
    .read_json('data/scraped_art_text.json')
    .T
    .reset_index()
    .rename({'index':'chapter', 'title':'chapter_title', 'link':'chapter_link'}
            ,axis='columns')
    .assign(chapter=lambda x: x.chapter.str.extract(r'(\d+)').astype('int'))
)

In [102]:
# empty df, loop will add rows to the bottom
long = pd.DataFrame()

for i in range(len(art_text.chapter)):
    unnested_articles = (
        pd
        .DataFrame(art_text.articles[i])
        .T
        .assign(chapter=i + 1)
    )
    
    sub_articles = (
        unnested_articles
        .assign(
            text=lambda x: x.text.str.replace(r'(\r\n|\r|\n)', '\t', regex=True)
               )
        .text.str.split(r'(?=\t\d*[^a-z1-9]).+?', expand=True)
        .apply(squeeze, axis=1)
        .T
        .melt(id_vars=None, value_name='text', var_name='article')
        .dropna()
        .assign(sub_art=lambda x: x.groupby('article').cumcount()+1)
        .reset_index(drop=True)
    )
        
    long_unnested = (
        unnested_articles
        .drop(columns='text')
        .reset_index()
        .merge(sub_articles, left_on='index', right_on='article')
        .loc[:,['chapter', 'title', 'link', 'article','sub_art', 'text']]
    )
    
    # finally add this to initialised df
    long=pd.concat([long, long_unnested], axis='rows')



In [133]:
unnested_articles = (
    pd
    .DataFrame(art_text.articles[0])
    .T
    .assign(chapter=0 + 1)
)


sub_articles = (
    unnested_articles
    .assign(
        text=lambda x: x.text.str.replace(r'(\r\n|\r|\n)', '\t', regex=True)
           )
    .text.str.split(r'(?=\t\d*[^a-z1-9]).+?', expand=True)
    .apply(squeeze, axis=1)
    .T

)

In [134]:
sub_articles

,1,2,3,4
0,This Regulation lays down rules relating to th...,This Regulation applies to the processing of p...,This Regulation applies to the processing of p...,‘personal data’ means any information relating...
1,This Regulation protects fundamental rights an...,This Regulation does not apply to the processi...,This Regulation applies to the processing of p...,‘processing’ means any operation or set of ope...
2,The free movement of personal data within the ...,\tin the course of an activity which falls out...,"\tthe offering of goods or services, irrespect...",‘restriction of processing’ means the marking ...
3,NaN,1For the processing of personal data by the Un...,This Regulation applies to the processing of p...,‘profiling’ means any form of automated proces...
4,NaN,This Regulation shall be without prejudice to ...,NaN,‘pseudonymisation’ means the processing of per...
5,NaN,NaN,NaN,‘filing system’ means any structured set of pe...
6,NaN,NaN,NaN,‘controller’ means the natural or legal person...
7,NaN,NaN,NaN,"‘processor’ means a natural or legal person, p..."
8,NaN,NaN,NaN,"1‘recipient’ means a natural or legal person, ..."
9,NaN,NaN,NaN,"‘third party’ means a natural or legal person,..."


In [95]:
(
    pd
    .DataFrame(art_text.articles[0])
    .T
    .assign(chapter=1 + 1)
    .text.str.replace(r'(\r\n|\r|\n)', r'\t', regex=True)
)[2]

'\tThis Regulation applies to the processing of personal data in the context of the activities of an establishment of a controller or a processor in the Union, regardless of whether the processing takes place in the Union or not.\tThis Regulation applies to the processing of personal data of data subjects who are in the Union by a controller or processor not established in the Union, where the processing activities are related to:\t\tthe offering of goods or services, irrespective of whether a payment of the data subject is required, to such data subjects in the Union; or\tthe monitoring of their behaviour as far as their behaviour takes place within the Union.\t\t\tThis Regulation applies to the processing of personal data by a controller not established in the Union, but in a place where Member State law applies by virtue of public international law.\t'

In [48]:
import re

In [97]:
example = '1\tTake a new line'
pattern = '((?=\\t\\d*[^a-z1-9]).+?)'

print(re.sub(pattern, '9', example))

19Take a new line


In [77]:
re.sub('-\n(?=[a-z])', '','-\nabc', re.M)

'abc'

In [82]:
txt = "1\nTake a new line"
x = re.sub(r"(\r\n|\r|\n)", "\t", txt, re.M)
y = re.sub(r"(?=\t\d*[^a-z1-9]).+?", "", x)
print(y)

1Take a new line


In [44]:
example.replace(pattern, '\\t')

'1\nTake a new line'

In [47]:
pattern

'(?=\\n\\d*[^a-z1-9]).+?'

sub sub articles !!! arg. need to look for lower case laters at the beginning of text. if so, add another bit a, b, ... required

In [180]:
import string

In [183]:
letters = [letter for letter in string.ascii_lowercase]

In [186]:
sub_articles

,article,text,sub_art
0,94,Directive 95/46/EC is repealed with effect fro...,1
1,94,1References to the repealed Directive shall be...,2
2,95,This Regulation shall not impose additional ob...,1
3,96,International agreements involving the transfe...,1
4,97,1By 25 May 2020 and every four years thereafte...,1
5,97,In the context of the evaluations and reviews ...,2
6,97,Chapter V on the transfer of personal data to ...,3
7,97,Chapter VII on cooperation and consistency.,4
8,97,"For the purpose of paragraph 1, the Commission...",5
9,97,In carrying out the evaluations and reviews re...,6
